# Dynamic lot-size model

This is a classic problem in balancing between holding inventory and setup costs. Originally proposed by Wagner & Whitin (1958).

$$
\begin{align}
    \text{min}~~ & \sum_{t \in T}{(h_{t} I_{t} + s_{t} y_{t})} \\
    \text{s.t.}~~ & I_{t} = I_{t - 1} + x_{t} - d_{t} & \forall ~ t \in T; t \geq 2\\
    & I_{1} = I_{0} + x_{1} - d_{1}\\
    & x_{t} \leq M y_{t} & \forall ~ t \in T \\
    & x_{t}; I_{t} \geq 0 & \forall ~ t \in T \\
    & y_{t} \in \left \{ 0, 1 \right \} & \forall ~ t \in T\\
\end{align}
$$

In [1]:
import numpy as np
import pandas as pd

from bnbprob.milpy import MILP
from bnbpy import BranchAndBound, configure_logfile

In [2]:
data = pd.read_csv("./../data/lot_size.csv")

## Write problem to matrix form

In [3]:
def create_lot_size(
    d: np.ndarray,
    s: np.ndarray,
    h: np.ndarray,
    **kwargs
) -> MILP:

    # Number of periods and big M
    T = len(d)
    M = sum(d)

    # 1. Cost vector: c = [0, ..., 0, h1, ..., hT, K1, ..., KT]
    # Production costs are zero in this basic model.
    c = np.concatenate([
        np.zeros(T),  # Production costs (assumed zero for now)
        h,  # Holding costs
        s,  # Setup costs
    ])

    # 2. Equality constraints (inventory balance): A_eq * x = b_eq
    A_eq = np.zeros((T, 3 * T))
    b_eq = np.array(d)  # Demand values

    for t in range(T):
        A_eq[t, t] = 1  # Coefficient of x_t in period t
        if t > 0:
            A_eq[t, T + t - 1] = 1  # Coefficient of I_{t-1} in period t
        A_eq[t, T + t] = -1  # Coefficient of I_t in period t

    # 3. Inequality constraints (setup cost constraints): A_ub * x <= b_ub
    A_ub = np.zeros((T, 3 * T))
    b_ub = np.zeros(T)  # Right-hand side is zero for all setup constraints

    for t in range(T):
        A_ub[t, t] = 1  # Coefficient of x_t in period t
        A_ub[
            t, 2 * T + t
        ] = -M

    x_ub = np.ones_like(c)
    x_ub[: 2 * T] = M
    x_ub[2 * T:] = 1.0

    def _idx_bounds(i: int):
        if i < 2 * T:
            return (0, None)
        return (0, 1)

    bounds = [_idx_bounds(i) for i in range(3 * T)]

    integrality = np.zeros_like(c)
    integrality[2 * T:] = 1.0

    milp = MILP(
        c,
        A_eq=A_eq,
        b_eq=b_eq,
        A_ub=A_ub,
        b_ub=b_ub,
        bounds=bounds,
        integrality=integrality,
        **kwargs
    )

    return milp

## Solve with bnbpy

In [4]:
milp = create_lot_size(
    data['demand'].values,
    data['setup_cost'].values,
    data['inventory_cost'].values,
    branching="max",
    seed=42
)

bnb = BranchAndBound(eval_node="out")

configure_logfile("lotsize.log", mode="w")

In [5]:
bnb.solve(milp, maxiter=20000)

Status: OPTIMAL | Cost: 864.0 | LB: 864.0

## Reference
Wagner, H. M., & Whitin, T. M. (1958). Dynamic version of the economic lot size model. Management science, 5(1), 89–96.